<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [2]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.5
Pandas:  1.0.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [0]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

In [6]:
bikes = pd.read_table('bikeshare.csv', header = None)
bikes.head()

,0
0,"instant,dteday,season,yr,mnth,hr,holiday,weekd..."
1,"1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,..."
2,"2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0..."
3,"3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0..."
4,"4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,..."


Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [15]:
#ANSWER:
bikes = pd.read_table('bikeshare.csv', header = 0, sep=',')
bikes.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [0]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [16]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [17]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name = 'Data', header = 0, usecols = [0, 1, 2, 3, 4])
df.head()

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [21]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [22]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                             Bank Name           City  ST   CERT  \
 0                 The First State Bank  Barboursville  WV  14361   
 1                   Ericson State Bank        Ericson  NE  18265   
 2     City National Bank of New Jersey         Newark  NJ  21111   
 3                        Resolute Bank         Maumee  OH  58317   
 4                Louisa Community Bank         Louisa  KY  58112   
 ..                                 ...            ...  ..    ...   
 556                 Superior Bank, FSB       Hinsdale  IL  32646   
 557                Malta National Bank          Malta  OH   6629   
 558    First Alliance Bank & Trust Co.     Manchester  NH  34264   
 559  National State Bank of Metropolis     Metropolis  IL   3815   
 560                   Bank of Honolulu       Honolulu  HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                         MVB Bank, Inc.      April 3, 2020  
 1             Farmers and Merchants Bank  F

In [29]:
df[0]

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date
0,The First State Bank,Barboursville,WV,14361,"MVB Bank, Inc.","April 3, 2020"
1,Ericson State Bank,Ericson,NE,18265,Farmers and Merchants Bank,"February 14, 2020"
2,City National Bank of New Jersey,Newark,NJ,21111,Industrial Bank,"November 1, 2019"
3,Resolute Bank,Maumee,OH,58317,Buckeye State Bank,"October 25, 2019"
4,Louisa Community Bank,Louisa,KY,58112,Kentucky Farmers Bank Corporation,"October 25, 2019"
...,...,...,...,...,...,...
556,"Superior Bank, FSB",Hinsdale,IL,32646,"Superior Federal, FSB","July 27, 2001"
557,Malta National Bank,Malta,OH,6629,North Valley Bank,"May 3, 2001"
558,First Alliance Bank & Trust Co.,Manchester,NH,34264,Southern New Hampshire Bank & Trust,"February 2, 2001"
559,National State Bank of Metropolis,Metropolis,IL,3815,Banterra Bank of Marion,"December 14, 2000"


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [10]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [69]:
#ANSWER
bikes.shape

(17379, 17)

What are the column names in `bikes`?

In [70]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [71]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [83]:
#ANSWER
for row in bikes.index:
    print(row, end='')

0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300301302303304305306307308309310311312313314315316317318319320321322323324325326327328329330331332333334335336337338339340341342343344345346347348349350351352353354355356357358359360361362363364365366367368369

2885288628872888288928902891289228932894289528962897289828992900290129022903290429052906290729082909291029112912291329142915291629172918291929202921292229232924292529262927292829292930293129322933293429352936293729382939294029412942294329442945294629472948294929502951295229532954295529562957295829592960296129622963296429652966296729682969297029712972297329742975297629772978297929802981298229832984298529862987298829892990299129922993299429952996299729982999300030013002300330043005300630073008300930103011301230133014301530163017301830193020302130223023302430253026302730283029303030313032303330343035303630373038303930403041304230433044304530463047304830493050305130523053305430553056305730583059306030613062306330643065306630673068306930703071307230733074307530763077307830793080308130823083308430853086308730883089309030913092309330943095309630973098309931003101310231033104310531063107310831093110311131123113311431153116311731183119312031213122312331243125312631273128312931303131313231333134

5845584658475848584958505851585258535854585558565857585858595860586158625863586458655866586758685869587058715872587358745875587658775878587958805881588258835884588558865887588858895890589158925893589458955896589758985899590059015902590359045905590659075908590959105911591259135914591559165917591859195920592159225923592459255926592759285929593059315932593359345935593659375938593959405941594259435944594559465947594859495950595159525953595459555956595759585959596059615962596359645965596659675968596959705971597259735974597559765977597859795980598159825983598459855986598759885989599059915992599359945995599659975998599960006001600260036004600560066007600860096010601160126013601460156016601760186019602060216022602360246025602660276028602960306031603260336034603560366037603860396040604160426043604460456046604760486049605060516052605360546055605660576058605960606061606260636064606560666067606860696070607160726073607460756076607760786079608060816082608360846085608660876088608960906091609260936094

9172917391749175917691779178917991809181918291839184918591869187918891899190919191929193919491959196919791989199920092019202920392049205920692079208920992109211921292139214921592169217921892199220922192229223922492259226922792289229923092319232923392349235923692379238923992409241924292439244924592469247924892499250925192529253925492559256925792589259926092619262926392649265926692679268926992709271927292739274927592769277927892799280928192829283928492859286928792889289929092919292929392949295929692979298929993009301930293039304930593069307930893099310931193129313931493159316931793189319932093219322932393249325932693279328932993309331933293339334933593369337933893399340934193429343934493459346934793489349935093519352935393549355935693579358935993609361936293639364936593669367936893699370937193729373937493759376937793789379938093819382938393849385938693879388938993909391939293939394939593969397939893999400940194029403940494059406940794089409941094119412941394149415941694179418941994209421

1188011881118821188311884118851188611887118881188911890118911189211893118941189511896118971189811899119001190111902119031190411905119061190711908119091191011911119121191311914119151191611917119181191911920119211192211923119241192511926119271192811929119301193111932119331193411935119361193711938119391194011941119421194311944119451194611947119481194911950119511195211953119541195511956119571195811959119601196111962119631196411965119661196711968119691197011971119721197311974119751197611977119781197911980119811198211983119841198511986119871198811989119901199111992119931199411995119961199711998119991200012001120021200312004120051200612007120081200912010120111201212013120141201512016120171201812019120201202112022120231202412025120261202712028120291203012031120321203312034120351203612037120381203912040120411204212043120441204512046120471204812049120501205112052120531205412055120561205712058120591206012061120621206312064120651206612067120681206912070120711207212073120741207512076120771207812079

1478414785147861478714788147891479014791147921479314794147951479614797147981479914800148011480214803148041480514806148071480814809148101481114812148131481414815148161481714818148191482014821148221482314824148251482614827148281482914830148311483214833148341483514836148371483814839148401484114842148431484414845148461484714848148491485014851148521485314854148551485614857148581485914860148611486214863148641486514866148671486814869148701487114872148731487414875148761487714878148791488014881148821488314884148851488614887148881488914890148911489214893148941489514896148971489814899149001490114902149031490414905149061490714908149091491014911149121491314914149151491614917149181491914920149211492214923149241492514926149271492814929149301493114932149331493414935149361493714938149391494014941149421494314944149451494614947149481494914950149511495214953149541495514956149571495814959149601496114962149631496414965149661496714968149691497014971149721497314974149751497614977149781497914980149811498214983

In [82]:
row_count = 0
for _ in bikes.index:
    row_count += 1 
print(f'There are {row_count} rows in this DataFrame')

There are 17379 rows in this DataFrame


In [30]:
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [84]:
#ANSWER
print(type(bikes['season']))

<class 'pandas.core.series.Series'>


How would we use object notation to show the first 4 rows of `atemp`?

In [85]:
#ANSWER
print(type(bikes[['season']]))

<class 'pandas.core.frame.DataFrame'>


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [32]:
#bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]
bikes[0:5]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


What is the correct way to access the 1st row of the DataFrame by its index?

In [33]:
#ANSWER
bikes.iloc[0:1]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [91]:
#ANSWER
bikes.iloc[:, 1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: looking for not a number values

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [92]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [93]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [34]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [40]:
#ANSWER
def division(data1, data2):
    if data2 == 0:
        result = np.nan
    else:
        result = data1 / data2
    return result

In [43]:
division(100,0)

nan

Apply the Pandas `isna` function to the following data objects:

In [96]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [98]:
#ANSWER
pd.isna(x), pd.isna(y)

(False, True)

In [99]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [100]:
#ANSWER
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

In [101]:
# Or turn array to DataFrame
arraydf = pd.DataFrame(array)
arraydf.isna()

,0,1,2
0,False,True,False
1,False,False,True


How is the pandas I/O parameter `na_values` used?

? ANSWER: Converting coded strings to NaN

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [107]:
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [104]:
#ANSWER
bikes['season'].value_counts().sort_index(ascending=True) #There are 4 seasons in bikes data set

1    4242
2    4409
3    4496
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [49]:
#ANSWER
def range(data):
    print(f'From {data.min()} to {data.max()}')
    

In [50]:
range(bikes.instant)



From 1 to 17379


In [115]:
print(range(bikes.windspeed))

From 0.0 to 0.8507


In [116]:
print(range(bikes.dteday))

From 2011-01-01 to 2012-12-31


Compute and print the overall minimum and maximum of the numeric data columns:

In [51]:
bikes_min, bikes_max = (min(bikes.min(numeric_only=True)), max(bikes.max(numeric_only=True)))
bikes_min, bikes_max

(0.0, 17379.0)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [52]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [53]:
#ANSWER
bikes['atemp'].quantile([0.1, 0.25, 0.5, 0.75, 0.9])

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

In [54]:
a = np.percentile(bikes['atemp'], [10, 25, 50, 75, 90])
print(a)

[0.2424 0.3333 0.4848 0.6212 0.697 ]


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [55]:
type(bikes['atemp'])

pandas.core.series.Series

In [56]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
14372,14373,2012-08-27,3,1,8,1,0,1,1,1,0.62,0.5606,0.88,0.0896,3,4,7
3388,3389,2011-05-26,2,0,5,2,0,4,1,1,0.62,0.5758,0.83,0.1045,7,7,14
6942,6943,2011-10-21,4,0,10,21,0,5,1,1,0.42,0.4242,0.58,0.1642,31,112,143
5219,5220,2011-08-10,3,0,8,9,0,3,1,1,0.76,0.6818,0.48,0.2239,32,148,180
5188,5189,2011-08-09,3,0,8,2,0,2,1,1,0.72,0.6818,0.70,0.0000,9,3,12


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [57]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [58]:
#ANSWER
type(atemp_level)

pandas.core.series.Series

Here is a random sample of `atemp_level`:

In [59]:
atemp_level.sample(5)          

11602    (0.5, 0.75]
9253     (0.25, 0.5]
7846     (0.5, 0.75]
16005    (0.25, 0.5]
5116     (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [132]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

10676    mild
7621     cool
14542    warm
15676    mild
2710     warm
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [133]:
bikes['atemp_level'] = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])

In [134]:
#ANSWER
result = bikes[(bikes['atemp_level'] == 'mild') & (bikes['season'] == 2)]
print(result['atemp_level'].count())

1829


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [139]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [140]:
def shape_value(data):
    print(f'Value:{data} Shape:{data.shape}')

In [141]:
shape_value(a)

Value:[0. 0. 0.] Shape:(3,)


In [142]:
shape_value(b)

Value:[[1. 1. 1.]] Shape:(1, 3)


In [143]:
shape_value(c)

Value:[[[1 9 6 7]
  [7 8 8 8]
  [6 3 4 1]]

 [[1 8 5 9]
  [1 5 1 6]
  [7 7 4 7]]] Shape:(2, 3, 4)


In [144]:
shape_value(d)

Value:[0 1 2 3] Shape:(4,)


In [145]:
shape_value(e)

Value:[[1 2 3 4]
 [5 6 7 8]] Shape:(2, 4)


## Load Data

Load rock.csv and clean the dataset.

In [3]:
rock = pd.read_csv('rock.csv')

## Check Column Names

Check column names and clean.

In [5]:
rock.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

In [6]:
rock.columns = rock.columns.str.lower().str.replace('?', '').str.replace('*', '').str.replace('clean','').str.strip()
rock.columns = rock.columns.str.replace(' ','_')

In [7]:
rock.columns

Index(['song', 'artist', 'release_year', 'combined', 'first', 'year',
       'playcount', 'fg'],
      dtype='object')

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [64]:
rock.isnull().sum()

song              0
artist            0
release_year    577
combined          0
first             0
year              0
playcount         0
fg                0
dtype: int64

In [ ]:
#Alternative, null_bool = rock['release_year'].isnull()
# rock.loc[null_bool, 'release_year'] = 0

In [8]:
rock.fillna(0, inplace=True)

In [9]:
rock.isnull().sum()

song            0
artist          0
release_year    0
combined        0
first           0
year            0
playcount       0
fg              0
dtype: int64

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [27]:
rock.dtypes

song            object
artist          object
release_year    object
combined        object
first            int64
year             int64
playcount        int64
fg               int64
dtype: object

In [10]:
rock['release_year'].unique()

array(['1982', 0, '1981', '1980', '1975', '2000', '2002', '1992', '1985',
       '1993', '1976', '1995', '1979', '1984', '1977', '1990', '1986',
       '1974', '2014', '1987', '1973', '2001', '1989', '1997', '1971',
       '1972', '1994', '1970', '1966', '1965', '1983', '1955', '1978',
       '1969', '1999', '1968', '1988', '1962', '2007', '1967', '1958',
       '1071', '1996', '1991', '2005', '2011', '2004', '2012', '2003',
       '1998', '2008', '1964', '2013', '2006', 'SONGFACTS.COM', '1963',
       '1961'], dtype=object)

In [11]:
mask = rock['release_year'] == 'SONGFACTS.COM' 
rock.loc[mask, 'release_year'] = np.NaN

In [12]:
rock['release_year'].unique()

array(['1982', 0, '1981', '1980', '1975', '2000', '2002', '1992', '1985',
       '1993', '1976', '1995', '1979', '1984', '1977', '1990', '1986',
       '1974', '2014', '1987', '1973', '2001', '1989', '1997', '1971',
       '1972', '1994', '1970', '1966', '1965', '1983', '1955', '1978',
       '1969', '1999', '1968', '1988', '1962', '2007', '1967', '1958',
       '1071', '1996', '1991', '2005', '2011', '2004', '2012', '2003',
       '1998', '2008', '1964', '2013', '2006', '1963', '1961'],
      dtype=object)

In [13]:
rock['release_year'] = pd.to_numeric(rock['release_year'])

In [14]:
rock.dtypes

song            object
artist          object
release_year     int64
combined        object
first            int64
year             int64
playcount        int64
fg               int64
dtype: object

In [15]:
rock.release_year.dtypes

dtype('int64')

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [18]:
def check_min_max(df):
    print(df.describe().T[['min', 'max']])
    

In [19]:
check_min_max(rock)

              min     max
release_year  0.0  2014.0
first         1.0     1.0
year          0.0     1.0
playcount     0.0   142.0
fg            0.0   142.0


In [20]:
mask1 = rock['release_year'] == 0 
rock.loc[mask1, 'release_year'] = np.NaN

In [21]:
check_min_max(rock)

                 min     max
release_year  1071.0  2014.0
first            1.0     1.0
year             0.0     1.0
playcount        0.0   142.0
fg               0.0   142.0


In [22]:
rock.release_year.unique()

array([1982.,   nan, 1981., 1980., 1975., 2000., 2002., 1992., 1985.,
       1993., 1976., 1995., 1979., 1984., 1977., 1990., 1986., 1974.,
       2014., 1987., 1973., 2001., 1989., 1997., 1971., 1972., 1994.,
       1970., 1966., 1965., 1983., 1955., 1978., 1969., 1999., 1968.,
       1988., 1962., 2007., 1967., 1958., 1071., 1996., 1991., 2005.,
       2011., 2004., 2012., 2003., 1998., 2008., 1964., 2013., 2006.,
       1963., 1961.])

# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [24]:
def draw_info(data):
    """Take a row index to show song, artist and release date"""
    print('Song Name:',  data['song'])
    print('Artist:',  data['artist'])
    print('Release date before 1970:',  data['release_year'] < 1970)

In [25]:
draw_info(rock.iloc[0])

Song Name: Caught Up in You
Artist: .38 Special
Release date before 1970: False


In [26]:
rock.head().apply(draw_info, axis=1)

Song Name: Caught Up in You
Artist: .38 Special
Release date before 1970: False
Song Name: Fantasy Girl
Artist: .38 Special
Release date before 1970: False
Song Name: Hold On Loosely
Artist: .38 Special
Release date before 1970: False
Song Name: Rockin' Into the Night
Artist: .38 Special
Release date before 1970: False
Song Name: Art For Arts Sake
Artist: 10cc
Release date before 1970: False


0    None
1    None
2    None
3    None
4    None
dtype: object

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [28]:
def convert_to_float(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

## Apply these functions to your dataset

In [30]:
rock.apply(convert_to_float)

,song,artist,release_year,combined,first,year,playcount,fg
0,NaN,NaN,1982.0,NaN,1,1,82,82
1,NaN,NaN,NaN,NaN,1,0,3,0
2,NaN,NaN,1981.0,NaN,1,1,85,85
3,NaN,NaN,1980.0,NaN,1,1,18,18
4,NaN,NaN,1975.0,NaN,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,NaN,NaN,1,0,1,0
2226,NaN,NaN,1981.0,NaN,1,1,32,32
2227,NaN,NaN,1975.0,NaN,1,1,109,109
2228,NaN,NaN,1983.0,NaN,1,1,1,1


## Describe the new float-only DataFrame.

In [33]:
rock.describe().T # Use .T or .transpose()

,count,mean,std,min,25%,50%,75%,max
release_year,1652.0,1978.019976,24.191247,1071.0,1971.0,1977.0,1984.0,2014.0
first,2230.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
year,2230.0,0.741256,0.438043,0.0,0.0,1.0,1.0,1.0
playcount,2230.0,16.872646,25.302972,0.0,1.0,4.0,21.0,142.0
fg,2230.0,15.048430,25.288366,0.0,0.0,3.0,18.0,142.0




> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



